# NER and Classification with LLMs

1. Install and configure OLLama with an appropriate LLM model (e.g. models from: Llama, Mistral, Bielik, Phi families). Rather not use models above 10B paramters. Sample LLM run command, when OLLama is installed: ollama run phi3:3.8b

In [1]:
import subprocess

def query_ollama(prompt):
    result = subprocess.run(
        ["C:\\Users\\gboch\\AppData\\Local\\Programs\\Ollama\\ollama.exe", "run", "phi3:3.8b"],
        input=prompt, capture_output=True, text=True
    )
    return result.stdout.strip() if result.returncode == 0 else None

prompt = "Odpowiedz krótko: Ile wynosi pierwiastek kwadratowy z 16?"
response = query_ollama(prompt)

print(response)

4


2. Take 1 thousand random passages from the FIQA-PL corpus. INFO: You can play with new dataset, but it will be necessary to create baseline results (next excersise).

In [2]:
from datasets import load_dataset
import pandas as pd

corpus = load_dataset("clarin-knext/fiqa-pl", 'corpus')
corpus_df = pd.DataFrame(corpus['corpus'])

corpus_1000 = corpus_df.sample(n=1000, random_state=30)
corpus_1000.head()

,_id,title,text
25281,261583,,Będziesz musiał pobrać gotówkę z konta karty k...
51486,535613,,Poza wydarzeniami z dnia na dzień i mechanizma...
43137,448791,,Jednym ze sposobów na zmniejszenie miesięcznej...
48863,507556,,> W wielu przypadkach faktycznie marnujemy ene...
12642,129947,,„**Zamierzam ponownie podkreślić ten post dla ...


3. As baseline use traditional NER methods from lab 7 - SpaCy.

In [3]:
import spacy

nlp = spacy.load("pl_core_news_sm")

def lemmatize(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

corpus_1000['text'] = corpus_1000['text'].astype(str)
corpus_1000['lemmas'] = corpus_1000['text'].apply(lemmatize).apply(pd.Series)

def extract_named_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return ', '.join(entities)

def spacy_large_ner(text):
  return ', '.join([ent.label_ for ent in nlp(text).ents])

corpus_1000['categories'] = corpus_1000['lemmas'].apply(spacy_large_ner).apply(pd.Series)
corpus_1000['entities'] = corpus_1000['lemmas'].apply(extract_named_entities).apply(pd.Series)
corpus_1000.head()

,_id,title,text,lemmas,categories,entities
25281,261583,,Będziesz musiał pobrać gotówkę z konta karty k...,być musieć pobrać gotówka z konto karta kredyt...,placeName,maklerski
51486,535613,,Poza wydarzeniami z dnia na dzień i mechanizma...,poza wydarzenie z dzień na dzień i mechanizm a...,"orgName, orgName, persName","DCF, DCF, Celsjusz"
43137,448791,,Jednym ze sposobów na zmniejszenie miesięcznej...,jeden z sposób na zmniejszyć miesięczny spłata...,,
48863,507556,,> W wielu przypadkach faktycznie marnujemy ene...,> w wiele przypadek faktycznie marnować energi...,,
12642,129947,,„**Zamierzam ponownie podkreślić ten post dla ...,„ * * zamierzać ponownie podkreślić ten post d...,"persName, orgName, placeName, persName, persNa...","OP, HF, BB, Perla, Analyst, Perl, Perl, Perlu,..."


4. Design prompts for the LLM to:
- Identify named entities in text
- Classify them into predefined categories (person, organization, location, etc.)

5. Implement prompt variations to compare performance:
- Zero-shot prompting

In [4]:
print(corpus_1000.iloc[11]['text'])

na początek zdobądź tani, łatwy pakiet oprogramowania księgowego, taki jak podręczniki, i poproś sprzedawców, aby przeszkolili Cię w jego obsłudze i konfiguracji. 50k, które zainwestujesz w spółkę, będzie liczone jako opłacony kapitał zakładowy. wówczas wszelkie przyszłe wypłaty z konta firmowego będą widoczne jako pożyczka dla dyrektora.


In [ ]:
text =corpus_1000.iloc[4]['text']

zero_shot = f"""
Zadanie: Znajdź wszystkie nazwane jednostki(NE) i sklasyfikuj je do jednej z kategorii: 
(placeName, orgName, persName, date, geogName, time, productName).
Text: {text}

Odpowiedź powinna być w formie poprawnego obiektu JSON, 
gdzie klucze to nazwane jednostki występujące w tekście, a wartości to kategoria
Żadne dodatkowe informacje, wyjaśnienia czy formatowanie nie są dołączane.
"""


response = query_ollama(zero_shot)
print(corpus_1000.iloc[4]['text'])
print("Model:\n", response)

„**Zamierzam ponownie podkreślić ten post dla OP** > Ponadto umiejętności sprzedażowe są kluczowe w tym biznesie, nawet w badaniach, ponieważ jeśli jesteś analitykiem po stronie sprzedaży, połowa Twojej pracy będzie dotyczyć klientów (tj. sprzedawać klientom) ze sprzedawcami. Zgodziłem się. Tak bardzo się zgodziłem. Zbyt wiele osób uważa, że ​​stanowiska badawcze w finansach są jak akademia albo jakieś gówno. Kto według nich płaci za badania i jaką rolę ich zdaniem jest sprzedaż badania? Na pewno nie wiceprezesów ani dyrektorów zarządzających. Co ważniejsze, umiejętności sprzedażowe są nieodłączną częścią sukcesu. Sednem sprzedaży jest komunikacja. Jeśli nie możesz sprzedać swoich badań, w jaki sposób zamierzasz sprzedać się na tyle dobrze, aby wiceprezesa/dyrektora generalnego lub obsadzać stanowisko w HF zajmującym się badaniami. > czy jest miejsce na wymówkę „ale nie jestem w tym dobry!”. Ludzie, którzy mają takie nastawienie, wychodzą za drzwi tak szybko, jak oni dokończ to zdanie 

- Few-shot prompting with 3-5 examples

In [3]:
text = corpus_1000.iloc[4]['text']

few_shot = f"""
Zadanie: Znajdź wszystkie nazwane jednostki (NE) w poniższym tekście i przypisz je do jednej z kategorii: 
(placeName, orgName, persName, date, geogName, time, productName). Odpowiedź powinna być w formie obiektu JSON, gdzie:

- **placeName**: to nazwy miejsc (np. miast, krajów),
- **orgName**: to nazwy organizacji (np. firm, instytucji),
- **persName**: to imiona i nazwiska osób,
- **date**: to daty,
- **geogName**: to nazwy regionów lub kontynentów,
- **time**: to godziny lub okresy czasowe,
- **productName**: to nazwy produktów, jeśli występują w tekście.

Proszę o podanie odpowiedzi zgodnie z poniższymi przykładami:

Przykład 1:
Tekst: "Anna Kowalska pracuje w firmie XYZ w Krakowie. Spotkanie odbyło się 15 marca 2023 roku."
Odpowiedź:
(
"placeName": ["Kraków"],
    "orgName": ["XYZ"],
    "persName": ["Anna Kowalska"],
    "date": ["15 marca 2023"],
    "geogName": [],
    "time": []
)

Przykład 2:
Tekst: "W Warszawie 20 stycznia 2022 roku odbył się ważny wykład. Prelegentem był Jan Nowak z Uniwersytetu Warszawskiego."
Odpowiedź:
(
    "placeName": ["Warszawa"],
    "orgName": ["Uniwersytet Warszawski"],
    "persName": ["Jan Nowak"],
    "date": ["20 stycznia 2022"],
    "geogName": [],
    "time": []
)

Przykład 3:
Tekst: "Zespół badawczy IBM zaprezentował nową technologię w Londynie, 5 czerwca 2021 roku."
Odpowiedź:
(
    "placeName": ["Londyn"],
    "orgName": ["IBM"],
    "persName": [],
    "date": ["5 czerwca 2021"],
    "geogName": [],
    "time": []
)

Przykład 4:
Tekst: "Paryż jest stolicą Francji, a jego mieszkańcy obchodzą 14 lipca, czyli Dzień Bastylii."
Odpowiedź:
(
    "placeName": ["Paryż"],
    "orgName": ["Francja"],
    "persName": [],
    "date": ["14 lipca"],
    "geogName": [],
    "time": []
)

Przykład 5:
Tekst: "iPhone 12 został wydany 23 października 2020 roku przez Apple w Nowym Jorku."
Odpowiedź:
(
    "placeName": ["Nowy Jork"],
    "orgName": ["Apple"],
    "persName": [],
    "date": ["23 października 2020"],
    "geogName": [],
    "time": [],
    "productName": ["iPhone 12"]
)

Proszę teraz wykonać to samo zadanie dla poniższego tekstu:

Tekst: {text}

Odpowiedź powinna być w takim samym formacie jak powyższe przykłady.
"""

In [7]:
prompt = few_shot
response = query_ollama(prompt)
print("Source text:", corpus_1000.iloc[4]['text'])
print("Model response:\n", response)

Source text: „**Zamierzam ponownie podkreślić ten post dla OP** > Ponadto umiejętności sprzedażowe są kluczowe w tym biznesie, nawet w badaniach, ponieważ jeśli jesteś analitykiem po stronie sprzedaży, połowa Twojej pracy będzie dotyczyć klientów (tj. sprzedawać klientom) ze sprzedawcami. Zgodziłem się. Tak bardzo się zgodziłem. Zbyt wiele osób uważa, że ​​stanowiska badawcze w finansach są jak akademia albo jakieś gówno. Kto według nich płaci za badania i jaką rolę ich zdaniem jest sprzedaż badania? Na pewno nie wiceprezesów ani dyrektorów zarządzających. Co ważniejsze, umiejętności sprzedażowe są nieodłączną częścią sukcesu. Sednem sprzedaży jest komunikacja. Jeśli nie możesz sprzedać swoich badań, w jaki sposób zamierzasz sprzedać się na tyle dobrze, aby wiceprezesa/dyrektora generalnego lub obsadzać stanowisko w HF zajmującym się badaniami. > czy jest miejsce na wymówkę „ale nie jestem w tym dobry!”. Ludzie, którzy mają takie nastawienie, wychodzą za drzwi tak szybko, jak oni dokoń

6. Compare results between:
- Traditional NER (SpaCy)
- Pure LLM-based approach

In [8]:
entities = corpus_1000['entities'].iloc[4].split(', ')
categories = corpus_1000['categories'].iloc[4].split(', ')

for entity, category in zip(entities, categories):
    print(f"'entity': '{entity}', 'category': '{category}'")

'entity': 'OP', 'category': 'persName'
'entity': 'HF', 'category': 'orgName'
'entity': 'BB', 'category': 'placeName'
'entity': 'Perla', 'category': 'persName'
'entity': 'Analyst', 'category': 'persName'
'entity': 'Perl', 'category': 'persName'
'entity': 'Perl', 'category': 'orgName'
'entity': 'Perlu', 'category': 'placeName'
'entity': 'Wall St .', 'category': 'geogName'
'entity': 'OP', 'category': 'persName'


8. Build a simple evaluation pipeline:
- Manually annotate 20 passages for ground truth (ideally, share those annotated passages in the group, so everyone have much more than 20)
- Compute precision, recall, and F1 score for each approach
- Analyze error patterns and classification mistakes

In [7]:
pd.set_option('display.max_colwidth', None)
corpus_20 = corpus_1000.sample(n=20, random_state=30)
display(corpus_20[["text"]])

,text
39464,"Aby odpowiedzieć na swoje pytanie, musisz zadać sobie pytanie. Powszechne koszty transakcyjne mogą być naprawdę trudne do zrekompensowania w ciągu jednego roku. Może to obejmować inspekcje domów, koszty zamknięcia, prowizje agentów itp. – razem może to być do 6-10% wartości Twojego domu. Jest to trudny cel do pokonania w ciągu roku, a marża na błędne obliczenia i wahania na rynku jest bardzo niska. Krótko mówiąc, możesz być wkurzony. Aby zarobić w ciągu roku, musisz zredukować koszty transakcyjne do minimum: Unikaj agentów, inspektorów, pośredników hipotecznych itp., którzy mogą się odwdzięczyć ciekawą niespodzianką. Podsumowując, kupowanie domu na rok może mieć sens tylko wtedy, gdy możesz zredukować wszystkie związane z tym koszty transakcji, robiąc je samodzielnie. Jeśli na rynku jest wiele domów na sprzedaż, to starałbym się przekonać kogoś, żeby wynajął dom na rok w jak najlepszych warunkach (a może nawet spróbował podnająć część pokoi) lub też na wynajem. właścicielem domu. W ten sposób unikniesz kosztów transakcyjnych z góry i będzie miał większy sens finansowy dla guru niebędącego guru nieruchomości."
57574,"Międzynarodowe środki z całego świata. W Stanach Zjednoczonych fundusz zagranicznych akcji byłby akcjami spoza USA. Znam dziwny trzeci wybór: fundusz amerykańskich firm, które czerpią sprzedaż głównie z zagranicy."
3834,"Myślę, że twoje zrozumienie jest prawidłowe, o ile to opisujesz, ale nie wspominasz mi o krytycznym szczególe. Sugerujesz również pewne szczegóły kary za wcześniejszą wypłatę / anulowanie, ale nie podajesz tych warunków szczegółowo. Gdzie i kiedy wypłacane są odsetki? Czy trafia na tę samą płytę CD do składania? Czy jest wypłacany na inne konto? Opis mówi, że jest on wyceniony na równi, więc przynajmniej wiemy, że odsetki nie muszą pozostawać niespłacone w ramach CD do czasu zapadalności, ale oznacza to również, że niekoniecznie otrzymujesz kapitalizację według stawki CD. Nie wiedząc, dokąd idzie oprocentowanie i jeśli jest ono dostępne do kapitalizacji, zachowaj ostrożność w porównaniu z innymi płytami CD / kontami oszczędnościowymi. Struktura dająca się składać może być lepszą opcją, nawet przy niższym RRSO."
49813,"Przypuszczam, że w zyskach kapitałowych powinna istnieć jakaś korekta o inflację. W ten sposób ci, którzy wykorzystują krótkoterminową zmienność rynku i zarabiają na inwestowaniu w nieruchomości, będą traktowani inaczej niż babcia, która mieszka w swoim domu od 30 lat. Myślę, że dlatego nazywają inflację niewidzialnym podatkiem."
35068,">Fed Up był całkiem niezły. Również Sugar Coated to jeden z moich ulubionych artykułów kulinarnych. Tak, z wyjątkiem tego, że cukier nie jest dobry (szczególnie dla zębów), to sam nie jest odpowiedzialny za pogorszenie stanu zdrowia populacji zachodniej. Możesz pobrać próbkę tkanki tłuszczowej od ludzi i poprzez badania laboratoryjne stwierdzić, skąd pochodzi, i to w przeważającej mierze nie z węglowodanów, ale z tłuszczu pokarmowego (zwierzęcego lub roślinnego)."
28741,"Kilkakrotnie rozbudowywaliśmy naszą obecną lokalizację. Zaczęliśmy od zaledwie 33 stacji, w miniony piątek w sklepie grało 80 osób. Jeśli masz na myśli franchising, to jest to ogromne ryzyko i rzadko działa tak, jak byś tego oczekiwał. Byłbym bardziej zainteresowany konsultacjami i w razie potrzeby byciem na miejscu, aby pomóc w pewnych sprawach, ale nie miałem jeszcze nikogo, kto mi zapłaci, a przynajmniej nikt, kto mi zapłacił."
11816,"Nie, nie. Usuwasz ofertę pracy w tym kraju i wysyłasz ją do innego. Ludzie nie mieliby nic przeciwko, gdyby dano im możliwość kontynuowania pracy w nowym kraju z kosztami utrzymania obniżonymi o nowe wynagrodzenie. Prawdopodobnie niewielu wybrałoby opcję, ale przynajmniej nie byłbyś SOL."
2268,"Jestem stary. Kiedy byłem młody, szkoły przygotowywały nas wszystkich do systemu metrycznego. Ale zmiana nigdy nie nastąpiła z powodu, jak sądzę, prostej arogancji. Niech świat poradzi sobie bez nakrętek i śrub 45/17. Oczywiście nie jesteśmy tak pot

In [4]:
def query_llm(text, prompt):
    prompt = prompt.format(text=text)
    response = query_ollama(prompt)
    return response

In [5]:
ground_truth = [
    {
        "orgName": ["RRSO", "SOL", "Toyoty", "Bayera", "Hondy", "AAPL", "AMT", "ISO", "TMT", 
                    "Microsoft", "Nadella", "Windows", "Linux", "IOS", "Chromium", "ETF-y", "ETF"],
        "persName": ["Fed Up", "Sugar Coated", "David Siegel", "Steve'owi Ballmerowi"],
        "date": ["2017 roku", "20 lat temu", "80-tymi", "90-tymi", "30 lat"],
        "geogName": ["Stanach Zjednoczonych", "USA", "Amsterdamie", "Ameryką", "Meksykanie", "Francuzki/Niemki"],
        "placeName": [],
        "time": []
    }
]

In [10]:
predictions_llm_zero = []
predictions_llm_few = []

for item in corpus_20['text']:
    text = item

    llm_zero_ents = query_llm(text, zero_shot)
    llm_few_ents = query_llm(text, few_shot)

    predictions_llm_zero.append(llm_zero_ents)
    predictions_llm_few.append(llm_few_ents)
    print(predictions_llm_zero)
    print(predictions_llm_few)

['{\n    "placeName": ["nawet"],\n    "orgName": [],\n    "persName": [],\n    "date": [],\n    "geogName": [],\n    "time": []\n}']
['(\n    "placeName": ["New York"],\n    "orgName": [],\n    "persName": ["John Smith", "Jane Doe"],\n    "date": ["today\'s date"], // The exact current date should be inserted here based on when the response is generated.\n    "geogName": ["Wall Street"],\n    "time": [],\n    "productName": []\n)']
['{\n    "placeName": ["nawet"],\n    "orgName": [],\n    "persName": [],\n    "date": [],\n    "geogName": [],\n    "time": []\n}', '```json\n{\n  "placeName": ["dom"],\n  "orgName": [],\n  "persName": [],\n  "date": [],\n  "geogName": [],\n  "time": [],\n  "productName": []\n}\n```\nIn the provided text, we are tasked to identify named entities belonging specifically to the categories listed. The \'placeName\' contains references to living spaces ("dom"), which also represents a potential real estate product name (\'realEstateProduct\'). Based on this info

In [ ]:
import spacy

predictions_spacy = []
nlp = spacy.load("pl_core_news_sm")

for item in corpus_20['text']:
    text = item
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    predictions_spacy.append(entities)
    print(entities)

[]
[('Stanach Zjednoczonych', 'placeName'), ('USA', 'placeName'), ('amerykańskich', 'placeName')]
[]
[]
[('Sugar Coated', 'geogName')]
[]
[('SOL', 'orgName')]
[('amerykańskie', 'placeName'), ('Toyoty', 'orgName'), ('Amsterdamie', 'placeName')]
[('Ameryką', 'placeName')]
[]
[('USA', 'placeName'), ('Bayera', 'persName'), ('Hondy', 'persName')]
[('Meksykanie', 'persName')]
[('rosyjski', 'placeName'), ('ukraiński', 'placeName'), ('mołdawski', 'persName'), ('białoruski', 'placeName'), ('Francuzki', 'placeName'), ('Niemki', 'placeName'), ('studencką', 'placeName'), ('studencką', 'placeName')]
[('AAPL', 'placeName'), ('2017 roku', 'date')]
[]
[('David Siegel', 'persName')]
[]
[('AMT', 'orgName'), ('TMT', 'orgName'), ('AMT', 'orgName'), ('AMT', 'orgName')]
[("Steve'owi", 'persName'), ('Ballmerowi Microsoft', 'persName'), ('Nadella', 'persName')]
[('ETF', 'orgName'), ('ETF', 'orgName'), ('ETF', 'orgName')]


In [38]:

zero_shot_dict = {"placeName": ["New York", "Warschau", "W Wall St.", "BB", "W Wall St", "Warszawa", "Nowy Jork", "College", "W Wall Street", "New York City"],
   "orgName": [], 
   "persName": ["John Smith", "Jane Doe", "Jan Nowak", "David Siegel", "Perła", "OP", "Zamierzam", "ona OP", "officer generalny", "dyrektor", 
                "HF", "Quant Analyst", "Perlem", "college students who took computer courses", "Pracującego", 
                "Anna Kowalska", "Jan Nowak", 
                "Perla",
                "Tego klienta", "Someone who studied in the college", "someone", "Anonymous", "ktoś", "IBM", "Apple\'s New York office",
                "Mieszko Brzostek", "Jakub Nowakowski", "Jan Mazurkiewicz", "Piotr Szymanski", "kto-średnia-posługiwać się Perlem", 
                "Quant Analyst", "wybudowający kodowanie w Perlu",
                "o", "nauczyciele", "Alice Smith","Jednego z tylu osób"],
   "date": ["today\'s date", "20 stycznia 2022", "na tyle długiej", "w momencie", "15 marca 2023", "20 czerwca 2021", "", "nieznany",
            "na mocy", "dzień", "miesiąc", "rok", "20 listopada 2023", "5 czerwca 2021", "23 października 2020", "5 czerwca 2021 roku",
            "23 października 2020 roku", "na mocy dnia", "dane w tekście"],  
   "geogName": ["Wall Street", "W Wall Street", "United States of America", "Wall St.", "W Wall St."],
   "time": ["godziny"],
   "productName": ["iPhone 12"],  }
                                                                                                                                                                                                                                                                                                                                                                                                                                            
few_shot_dict = {"placeName": ["nawet", "dom", "domu", "miejsdo", "kupować domu", "domu na rynku", "home"],
            "orgName": ["Windows", "Linux", "SOL", "Toyoty"],
            "persName": ["John Smith", "Jane Doe", "David Siegel", "guru"],
            "date": ['2017 roku', "rok", "ciągi",  "marża na błędne obliczenia i wahania na rynku", "mowa o tym roku", "today", "in the current year", 
                     "during this financial year"],
            "geogName": ["USA", "Stanach Zjednoczonych","zredukowanie kosztu transakcji", 
                         "kupowanie domu na rok", "podsumowując"],
            "time": ["góry", "mówiąc kro Pennsylvania"],
            "productName": ["realEstateProduct", "domek"] }


spacy_dict = {"placeName": ['Stanach Zjednoczonych', 'USA', 'amerykańskich', 'amerykańskie', 'Amsterdamie', 'Ameryką','rosyjski', 'ukraiński',
                             'białoruski', 'Francuzki', 'Niemki', 'studencką', 'AAPL'],
            "orgName": ['SOL', 'Toyoty', 'AMT', 'TMT', 'ETF'],
            "persName": ['Bayera', 'Hondy', 'Meksykanie', 'mołdawski', 'David Siegel', "Steve'owi", 'Ballmerowi Microsoft', 'Nadella'],
            "date": ['2017 roku'],
            "geogName": ['Sugar Coated'],
            "time": [],
            "productName": [] }

ground_truth = {
        "orgName": ["RRSO", "SOL", "Toyoty", "Bayera", "Hondy", "AAPL", "AMT", "ISO", "TMT", 
                    "Microsoft", "Nadella", "Windows", "Linux", "IOS", "Chromium", "ETF-y", "ETF"],
        "persName": ["David Siegel", "Steve'owi Ballmerowi"],
        "date": ["2017 roku", "20 lat temu", "80-tymi", "90-tymi", "30 lat"],
        "geogName": ["Stanach Zjednoczonych", "USA", "Amsterdamie", "Ameryką", "Meksykanie", "Francuzki", "Niemki", 'mołdawski', 
                     'rosyjski', 'ukraiński', 'białoruski'],
        "placeName": [],
        "time": [],
        "productName": ["Fed Up", "Sugar Coated"]
    }

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_global_metrics(predictions, ground_truth):
    # Połącz predykcje i prawdziwe etykiety w jeden zbiór
    all_preds = [item for sublist in predictions.values() for item in sublist]
    all_truth = [item for sublist in ground_truth.values() for item in sublist]
    
    all_preds = list(set(all_preds))
    all_truth = list(set(all_truth))
    
    TP = len(set(all_preds).intersection(all_truth))
    FP = len(set(all_preds) - set(all_truth))
    FN = len(set(all_truth) - set(all_preds))
    
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    
    return {"Precision": precision, "Recall": recall, "F1 Score": f1}

In [35]:
# Obliczanie metryk globalnie
global_metrics_zero = calculate_global_metrics(zero_shot_dict, ground_truth)
print("Global metrics for zero_shot:", global_metrics_zero)

Global metrics for zero_shot: {'Precision': 0.14705882352941177, 'Recall': 0.2702702702702703, 'F1 Score': 0.19047619047619047}


In [39]:
global_metrics_few = calculate_global_metrics(few_shot_dict, ground_truth)
print("Global metrics for few_shot:", global_metrics_few)

global_metrics_spacy = calculate_global_metrics(spacy_dict, ground_truth)
print("Global metrics for spacy:", global_metrics_spacy)

Global metrics for few_shot: {'Precision': 0.25, 'Recall': 0.21621621621621623, 'F1 Score': 0.2318840579710145}
Global metrics for spacy: {'Precision': 0.8214285714285714, 'Recall': 0.6216216216216216, 'F1 Score': 0.7076923076923075}


## Questions (2 points):

1. How does the performance of LLM-based NER compare to traditional approaches? What are the trade-offs in terms of accuracy, speed, and resource usage?

Porównanie LLM z tradycyjnymi metodami NER: LLM oferują wyższą dokładność, szczególnie w złożonych przypadkach, ale są wolniejsze i bardziej zasobożerne. Tradycyjne metody są szybsze i mniej wymagające, ale mogą mieć gorszą wydajność w trudniejszych zadaniach.

2. Which prompting strategy proved most effective for NER and classification tasks? Why?

Najbardziej efektywna strategia promptowania: Skuteczne jest promptowanie kontekstowe, które precyzyjnie wskazuje, jakiego rodzaju informacje mają być wyodrębnione, np. "Wskaż nazwy firm". Takie podejście poprawia precyzyjność zarówno w NER, jak i klasyfikacji.

3. What are the limitations and potential biases of using LLMs for NER and classification?

Ograniczenia i uprzedzenia LLM: LLM mogą przejmować uprzedzenia z danych treningowych, a także mają trudności z rozpoznawaniem specyficznych terminów branżowych. Mogą też generować wyniki, które są logiczne, ale nie zawsze trafne.

4. In what scenarios would you recommend using traditional NER vs. LLM-based approaches?

Kiedy używać tradycyjnego NER: Tradycyjne metody są lepsze, gdy mamy proste zadania z dobrze zdefiniowanymi bytami, ograniczone zasoby obliczeniowe lub wymaganą wysoką precyzję w specyficznych dziedzinach, np. medycynie.